In [ ]:
# Create submission script for 2024 by using the receivals from 2024

import pandas as pd
import numpy as np

orders_with_receivals = pd.read_csv("../data_cleaned/orders_with_receivals_detailed.csv", parse_dates=["delivery_date", "created_date_time", "modified_date_time", "date_arrival"])
orders_with_receivals_2024 = orders_with_receivals[orders_with_receivals['date_arrival'].dt.year == 2024]
orders_with_receivals_2024 = orders_with_receivals_2024[['rm_id', 'date_arrival', 'net_weight']]

In [ ]:
sample_submission = pd.read_csv("../data/sample_submission.csv")
prediction_mapping = pd.read_csv("../data/prediction_mapping.csv", parse_dates=["forecast_start_date", "forecast_end_date"])
submission = sample_submission.merge(prediction_mapping, on="ID")

In [ ]:
print(sample_submission.shape)
print(prediction_mapping.shape)
print(submission.shape)

(30450, 2)
(30450, 4)
(30450, 5)


In [ ]:
# For every forecast_start_date and forecast_end_date in submission make it 2024 instead of 2025
submission['forecast_start_date'] = submission['forecast_start_date'].apply(lambda x: x.replace(year=2024))
submission['forecast_end_date'] = submission['forecast_end_date'].apply(lambda x: x.replace(year=2024))

In [ ]:

for receival in orders_with_receivals_2024.itertuples():
    rm_id = receival.rm_id
    date_arrival = receival.date_arrival
    net_weight = receival.net_weight
    # Convert date_arrival to naive datetime for comparison
    date_arrival_naive = date_arrival.replace(tzinfo=None)
    submission.loc[
        (submission['rm_id'] == rm_id) & (submission['forecast_end_date'] >= date_arrival_naive),
        'predicted_weight'
    ] += net_weight

In [ ]:
# save submission to a csv
submission.to_csv("./submission_2024.csv", index=False)

In [ ]:
# Ehhrm some RM_IDS were not there so I wanted to double check .itertuples(), but I just found out that some RM_IDs are seasonal
valid_rm_ids1 = list(orders_with_receivals_2024['rm_id'].unique())
valid_rm_ids2 = list(orders_with_receivals_2024['rm_id'].unique())


count = 0

for receival in orders_with_receivals_2024.itertuples():
    rm_id = receival.rm_id
    date_arrival = receival.date_arrival
    net_weight = receival.net_weight
    # Convert date_arrival to naive datetime for comparison
    if rm_id in valid_rm_ids1:
        valid_rm_ids1.remove(rm_id)
    if rm_id not in valid_rm_ids2:
        count += 1
        print(f"RM_ID {rm_id} not found!")
    

print(count)

RM_ID nan not found!
1


In [ ]:
print(valid_rm_ids1)
print(valid_rm_ids2)

[np.float64(nan)]
[np.float64(3883.0), np.float64(2130.0), np.float64(2147.0), np.float64(2129.0), np.float64(3581.0), np.float64(3265.0), np.float64(3122.0), np.float64(3121.0), np.float64(3901.0), np.float64(2124.0), np.float64(2741.0), np.float64(3125.0), np.float64(3126.0), np.float64(3124.0), np.float64(3282.0), np.float64(3123.0), np.float64(2140.0), np.float64(3761.0), np.float64(2132.0), np.float64(3865.0), np.float64(3781.0), np.float64(2131.0), np.float64(2145.0), np.float64(2135.0), np.float64(3421.0), np.float64(2144.0), np.float64(2142.0), np.float64(2143.0), np.float64(2134.0), np.float64(2133.0), np.float64(nan), np.float64(3621.0), np.float64(2981.0), np.float64(3142.0), np.float64(4021.0), np.float64(4044.0), np.float64(3642.0), np.float64(3362.0), np.float64(3381.0), np.float64(2125.0), np.float64(4161.0), np.float64(2161.0), np.float64(4081.0), np.float64(4222.0), np.float64(4263.0), np.float64(3601.0), np.float64(3201.0), np.float64(4302.0), np.float64(4401.0), np.f

In [ ]:
test_submission = pd.read_csv("./submission_2024.csv")
test_submission = test_submission[["ID", "predicted_weight"]]
test_submission["predicted_weight"] = test_submission["predicted_weight"] * 0.7
test_submission.to_csv("./test2024.csv", index=False)